In [9]:
import pandas as pd
import feather 

# TODO:
# - keep patientnrs (add for blood bank)

# Load data

In [10]:
other_imid = pd.read_spss("../data/interim/OtherIMID.sav") 
all = pd.read_spss("../data/interim/Alles 1e sample.sav") # SLE patients (also has non- and other-IMID data)
non_imid = pd.read_spss("../data/interim/Non-Imid control set.sav")
tmo = pd.read_csv("../data/interim/TMO.csv") # blood bank controls (also has data from SLE patients)

# load translation table of columns in TMO.csv vs. the .sav files
df_cols = pd.read_csv("../references/chip_colnames.csv", sep=";")  

# Process

In [11]:
sle = all[all.SLE == 1] # keep only SLE patientsb
blood_bank = tmo[tmo.Class == "nonSLE"] # keep only blood baknk controls

The chip columns are called differently in the blood_bank dataset than in the others. So we'll rename the columns there

In [12]:
df_cols = df_cols.dropna() # keep only names of columns that exist in both data sets
new_colnames = df_cols.TB.tolist() # keep names as in .sav files
new_colnames.append('Class') # extra column in blood bank data (we'll need it in the others as well)
blood_bank.columns = new_colnames # rename columns as in other datasets

Then, for the others, keep only the TF chip data (i.e. the columns in the blood bank dataset

In [14]:
other_imid['Class'] = "IMID" 
other_imid = other_imid.loc[:,new_colnames] # keep only columns that also occur in blood_bank

non_imid['Class'] = "nonIMID"
non_imid = non_imid.loc[:,new_colnames]

sle['Class'] = "SLE"
sle = sle.loc[:,new_colnames]

And row-bind all the data frames together

In [24]:
# join all data frames together by binding rows
df_all = pd.concat([sle, other_imid, non_imid, blood_bank])

# Write data

In [23]:
feather.write_dataframe(df_all, "../data/processed/imid.feather")